# Setup

In [ ]:
# script for standard imports
%run '/content/drive/MyDrive/Data_Science/scripts/setup.ipynb'

In [ ]:
trx_nc_prepped = \
pd.read_csv('/content/drive/MyDrive/Data_Science/data/gtd/trx_nc_prepped.csv')

trx_nc_prepped = \
pd.read_csv('/content/drive/MyDrive/Data_Science/data/gtd/trainy.csv')

# Tuning model, Scoring & Evaluation
- Randomized Search and tuning parameters for selecting best model

In [ ]:
lr=LogisticRegression(random_state=33,n_jobs=-1)
tree=DecisionTreeClassifier(random_state=33)
forest=RandomForestClassifier(random_state=33,n_jobs=-1)
sgd=SGDClassifier(random_state=33,n_jobs=-1)

## Individual search spaces for each estimator (individual searches)

In [ ]:
logreg_grid = [{'penalty': ['l1', 'l2'],
                'C': np.logspace(0, 4, 10),
                'solver':['lbfgs', 'liblinear', 'sag', 'saga'],
                'class_weight':[None, 'balanced']}]

In [ ]:
dt_grid = [{'criterion': ['gini', 'entropy'],
            'max_depth': [5,10,20,None],
            'class_weight': [None, 'balanced'],
            'min_samples_split': [2,5,10],
            'max_features': ['auto', 'sqrt', 'log2'],
            'class_weight': [None, 'balanced']}]

In [ ]:
rf_grid = [{'n_estimators': [10, 50, 100, 200],
            'criterion': ['gini', 'entropy'],
            'max_depth': [5, 10, 20, None],
            'class_weight': [None, 'balanced'],
            'max_features': ['auto', 'sqrt', 'log2'],
            'bootstrap':[True,False]}]

In [ ]:
sgd_grid = [{'clf__class_weight':['balanced', None],
             'clf__penalty': ['l1','l2'],
             'clf__early_stopping': [True, False]}]

In [ ]:
 """complete grid all models"""
    # param_grid = [{'clf': [lr],
    #             'clf__penalty': ['l1', 'l2'],
    #             'clf__C': np.logspace(0, 4, 10),
    #             'clf__solver':['lbfgs', 'liblinear', 'sag', 'saga'],
    #             'clf__class_weight':[None, 'balanced']},
    #             {'clf':[tree],
    #             'clf__criterion': ['gini', 'entropy'],
    #             'clf__max_depth': [5,10,20,None],
    #             'clf__class_weight': [None, 'balanced'],
    #             'clf__min_samples_split': [2,5,10],
    #             'clf__max_features': ['auto', 'sqrt', 'log2'],
    #             'clf__class_weight': [None, 'balanced']},
    #             {'clf': [forest],
    #             'clf__n_estimators': [10, 50, 100, 200],
    #             'clf__criterion': ['gini', 'entropy'],
    #             'clf__max_depth': [5, 10, 20, None],
    #             'clf__class_weight': [None, 'balanced'],
    #             'clf__max_features': ['auto', 'sqrt', 'log2'],
    #             'clf__bootstrap':[True,False]},
    #             {'clf': [sgd],
    #             'clf__class_weight':['balanced', None],
    #             'clf__penalty': ['l1','l2'],
    #             'clf__early_stopping': [True, False]}]

'complete grid all models'

In [ ]:
# # classification scoring metrics
# clf_metrics = [accuracy_score,precision_score,
#                recall_score,f1_score]

## Execute Randomized Search (per model)

###Logisitic Regression

In [ ]:
# cross-val method for model score/eval
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=0)

In [ ]:
lr.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [ ]:
print(f'Randomized Search for Logisitic Regression:\n\n')

# logreg_pipe=Pipeline([('model', lr)])
eval_methods=['accuracy','precision','recall','f1']

logreg_grid = [{'penalty': ['l1', 'l2'],
                'C': np.logspace(0, 4, 10),
                'solver':['lbfgs', 'liblinear', 'sag', 'saga'],
                'class_weight':[None, 'balanced']}]

#instantiate RSVC and fit to data
rsvc_logreg=[]
for metric in eval_methods:
    rsvc_logreg.append(
        RandomizedSearchCV(lr, logreg_grid, cv=cv, verbose=0,
                           scoring=metric,return_train_score=False,
                           n_jobs=-1).fit(trx_nc_prepped,trainy.values))
#fit/train
# rsvc_logreg.fit(trx_nc_prepped,trainy.values)

# logisitic regression best parameters identified by rscv
# for each in rsvc_logreg:
#     for metric in eval_methods:
#         print(f'Logisitic regression best paramaters identified by rscv:\n\
#         for {metric}: {each.best_params_}}\n\n')
"""if above doesnt work try w/:"""
# for each in rsvc_logreg:
#     print(f'Logisitic regression best paramaters identified by rscv:\n\
#     {each.best_params_}}\n\n')

Randomized Search for Logisitic Regression:




In [ ]:
lr_best_score_permetric=[]
lr_best_model_per_metric=[]
lr_feat_importances_per_metric=[]
logreg_mean_score_permetric=[]
logreg_std_score_permetric=[]

for metric in eval_methods:
    for each in rsvc_logreg:
        # view/get best paramaters/model per score-metric
        lr_best_score_permetric.append(
            {metric: [each.best_score_]})
        lr_best_model_per_metric.append(
            {metric: [each.best_estimator_]})
        lr_feat_importances_per_metric.append(
            {metric:[each.best_estimator_.feature_importances_]})
        """nested CV"""
        #mean score per metric for best model
        logreg_mean_score_permetric.append((str(metric)+'_mean', 
                                  cross_val_score(each, trx_nc_prepped, 
                                  trainy.values, scoring=metric).mean()))
        #standard-dev per metric for best model
        logreg_std_score_permetric.append((str(metric)+'_std',
                                 cross_val_score(each, trx_nc_prepped,
                                 trainy.values, scoring=metric).std()))

In [ ]:
# view best model and its best score
for each_score, each_best in zip(lr_best_score_permetric,
                                 lr_best_model_per_metric):
    print(f'Logistic Regression "best score": {each_score}\n')
    print(f'\nLogistic Regression "best estimator": {each_best}')
    # print(grid.best_params_)

In [ ]:
# plot the cv-score of model,
# as a function of candidate hp-values for [--hp of choice--]:
"""... used initially on gscv i.e. "rsvc_logreg.grid_scores_" ...
       so may not work on rscv?"""
rsvc_logreg_scores = [val.mean_validation_score for val in rsvc_logreg.grid_scores_]
plt.semilogx("HP-of-choice", rsvc_logreg_scores)
plt.xlabel("hp-name")
plt.ylabel('accuracy')
# 5'print(f'(default-score-setting) for Accuracy: {logreg_grid.best_params_)}')

### Decision Tree

In [ ]:
print(f'Randomized Search for Decision Tree:\n\n')

dt_pipe=Pipeline([('model', lr)])
metric_str_names=['accuracy','precision','recall','f1']

#instantiate RSVC
rsvc_dt= \
RandomizedSearchCV(dt_pipe, dt_grid, cv=3, verbose=0,
                   scoring=metric,return_train_score=False,
                   n_jobs=-1))

#fit/train
rsvc_dt.fit(trx_nc_prepped,trainy.values)

# view items
# rsvc_dt.get_params().keys()

# Decision Tree best paramaters identified by rscv
print(f'Decision Tree best paramaters identified by rscv:\n\
{rsvc_dt.best_params_}\n\n')
"""if above doesnt work try w/:"""
# print(f'Decision Tree best paramaters identified by rscv:\n\
# {dt_grid.best_params_}\n\n')


# nested CV
dt_mean_score=[]
dt_std_score=[]
for metric in metric_str_names:
    #mean score per metric for best model
    dt_mean_score.append((str(metric)+'_mean', 
            cross_val_score(rsvc_dt trx_nc_prepped, 
                            trainy.values, scoring=metric).mean()))
    #standard-dev per metric for best model
    dt_std_score.append((str(metric)+'_std',
            cross_val_score(rsvc_dt trx_nc_prepped, 
                            trainy.values, scoring=metric).std()))
# view/get best paramaters/model per score-metric
dt_best_score=[]
dt_best_model=[]
# features_importances=[]

for metric in metric_str_names:
    dt_best_score.append({metric: [rsvc_dt.best_score_]})
    dt_best_model.append({metric: [rsvc_dt.best_estimator_]})
    # features_importances.append({metric:[each.best_estimator_.feature_importances_]})

# view
print(f'Decision Tree "best score": {dt_best_score}\n')
print(f'\nDecision Tree "best estimator": {dt_best_model}')

# plot the cv-score of model,
# as a function of candidate hp-values for [--hp of choice--]:
"""... used initially on gscv i.e. "rsvc_logreg.grid_scores_" ...
       so may not work on rscv?"""
rsvc_dt_scores = [val.mean_validation_score for val in rsvc_dt.grid_scores_]
plt.semilogx("HP-of-choice", rsvc_dt_scores)
plt.xlabel("hp-name")
plt.ylabel('accuracy')
# print(f'(default-score-setting) for Accuracy: {rsvc_dt.best_params_)}')

### Random Forest


In [ ]:
print(f'Randomized Search for Random Forest:\n\n')

rf_pipe=Pipeline([('model', lr)])
metric_str_names=['accuracy','precision','recall','f1']

#instantiate RSVC
rsvc_rf = \
RandomizedSearchCV(rf_pipe, rf_grid, cv=3, verbose=0,
                   scoring=metric,return_train_score=False, n_jobs=-1))
#fit/train
rsvc_rf.fit(trx_nc_prepped,trainy.values)

# view all parameters
# rsvc_rf.get_params().keys()

# Random Forest best paramaters identified by rscv
print(f'Random Forest best paramaters identified by rscv:\n\
{rsvc_rf.best_params_}\n\n')
"""if above doesnt work try w/:"""
# print(f'Decision Tree best paramaters identified by rscv:\n\
# {rf_grid.best_params_}\n\n')

# nested CV
rf_mean_score=[]
rf_std_score=[]
for metric in metric_str_names:
    #mean score per metric for best model
    rf_mean_score.append((str(metric)+'_mean', 
            cross_val_score(rsvc_rf, trx_nc_prepped, 
                            trainy.values, scoring=metric).mean()))
    #standard-dev per metric for best model
    rf_std_score.append((str(metric)+'_std',
            cross_val_score(rsvc_rf, trx_nc_prepped, 
                            trainy.values, scoring=metric).std()))
# view/get best paramaters/model per score-metric
rf_best_score=[]
rf_best_model=[]
# features_importances=[]

for metric in metric_str_names:
    rf_best_score.append({metric: [rsvc_rf.best_score_]})
    rf_best_model.append({metric: [rsvc_rf.best_estimator_]})
    # features_importances.append({metric:[each.best_estimator_.feature_importances_]})

# view
print(f'Random Forest "best score": {rf_best_score}\n')
print(f'\nRandom Forest "best estimator": {rf_best_model}')

# plot the cv-score of model,
# as a function of candidate hp-values for [--hp of choice--]:
"""... used initially on gscv i.e. "rsvc_logreg.grid_scores_" ...
       so may not work on rscv?"""
rsvc_rf_scores = [val.mean_validation_score for val in rsvc_rf.grid_scores_]
plt.semilogx("HP-of-choice", rsvc_rf_scores)
plt.xlabel("hp-name")
plt.ylabel('accuracy')
# print(f'(default-score-setting) for Accuracy: {rsvc_dt.best_params_)}')

### SGD

In [ ]:
print(f'Randomized Search for Stochastic Gradient Descent (SGD):\n\n')

sgd_pipe=Pipeline([('model', lr)])
metric_str_names=['accuracy','precision','recall','f1']

#instantiate RSVC
rsvc_sgd = \
RandomizedSearchCV(sgd_pipe, sgd_grid, cv=3, verbose=0,
                   scoring=metric,return_train_score=False,
                   n_jobs=-1))

#fit/train
rsvc_sgd.fit(trx_nc_prepped,trainy.values)

# view items
# >>> rsvc_sgd.get_params().keys()

# SGD best paramaters identified by rscv
print(f'SGD best paramaters identified by rscv:\n\
{rsvc_sgd.best_params_}\n\n')
"""if above doesnt work try w/:"""
# print(f'Decision Tree best paramaters identified by rscv:\n\
# {sgd_grid.best_params_}\n\n')

# nested CVsgd_mean_score=[]sgd_std_score=[]
for metric in metric_str_names:
    #mean score per metric for best model
sgd_mean_score.append((str(metric)+'_mean', 
            cross_val_score(rsvc_sgd, trx_nc_prepped, 
                            trainy.values, scoring=metric).mean()))
    #standard-dev per metric for best model
sgd_std_score.append((str(metric)+'_std',
            cross_val_score(rsvc_sgd, trx_nc_prepped, 
                            trainy.values, scoring=metric).std()))
# view/get best paramaters/model per score-metric
sgd_best_score=[]
sgd_best_model=[]
# features_importances=[]

for metric in metric_str_names:#, rsvc_sgd):
    sgd_best_score.append({metric: [rsvc_sgd.best_score_]})
    sgd_best_model.append({metric: [rsvc_sgd.best_estimator_]})
    # features_importances.append({metric:[each.best_estimator_.feature_importances_]})

# view
print(f'SGD "best score": {sgd_best_score}\n\n')
print(f'\SGD "best estimator": {sgd_best_model}')

# plot the cv-score of model,
# as a function of candidate hp-values for [--hp of choice--]:
"""... used initially on gscv i.e. "rsvc_logreg.grid_scores_" ...
       so may not work on rscv?"""
rsvc_sgd_scores = [val.mean_validation_score for val in rsvc_sgd.grid_scores_]
plt.semilogx("HP-of-choice", rsvc_sgd_scores)
plt.xlabel("hp-name")
plt.ylabel('accuracy')
# print(f'(default-score-setting) for Accuracy: {rsvc_sgd.best_params_)}') # or try sgd_grid

In [ ]:
""" ---------------------Here for Reference------------------------"""

    # # -------------Feature Selection----------------------------------------
        # from sklearn.feature_selection import  chi2, f_classif, mutual_info_classif


    # # -------------- Dimensionality Reduction ------------------------------
        # from sklearn.decomposition import (PCA, KernelPCA,
        #                                    IncrementalPCA, 
        #                                    NMF, TruncatedSVD)
        # from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



    # # ----------------- Reg/Classif Algorithms --------------------------
        # from sklearn.linear_model import SGDRegressor, SGDClassifier
        # from sklearn.linear_model import LinearRegression, LogisticRegression
        # from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
        # from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
        # from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
        # from sklearn.svm import LinearSVR, SVR, LinearSVC, SVC


    # #-------------------- Model Scoring/Evaluation -------------------------
        # from sklearn.metrics import (accuracy_score,
        #                              precision_score,
        #                              recall_score,
        #                              f1_score,
        #                              mean_squared_error,
        #                              mean_absolute_error,
        #                              make_scorer,
        #                              confusion_matrix,
        #                              precision_recall_curve,
        #                              auc,
        #                              roc_curve,
        #                              roc_auc_score,
        #                              mutual_info_score,
        #                              multilabel_confusion_matrix)


    # #-------------------- Model Selection ---------------------------------
        # from sklearn.dummy import DummyClassifier
        # from sklearn.model_selection import cross_validate
        # from sklearn.model_selection import cross_val_score
        # from sklearn.model_selection import cross_val_predict
        # from sklearn.model_selection import learning_curve
        # from sklearn.model_selection import validation_curve
        # from sklearn.model_selection import permutation_test_score
        # from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
        # from sklearn.model_selection import ParameterGrid
        # from sklearn.model_selection import ParameterSampler

In [ ]:
"""Linear Regression modeling process examples"""

    # """LR"""
    #     # lrclf.fit(trx_nc_prepped, train_y)
    #     # lrclf_preds = lrclf.predict(trx_nc_prepped)
    #     # lrclf_mse = mean_squared_error(train_y, lrclf_preds)
    #     # lrclf_rmse = np.sqrt(lrclf_mse)
    #     # lrclf_mae = mean_absolute_error(housing_labels, housing_predictions)
    # """tree"""
    #     # tree_reg.fit(housing_prepared, housing_labels)
    #     # housing_predictions = tree_reg.predict(housing_prepared)
    #     # tree_mse = mean_squared_error(housing_labels, housing_predictions)
    #     # tree_rmse = np.sqrt(tree_mse)
    # """forest"""
    #     # forest_reg.fit(housing_prepared, housing_labels)
    #     # housing_predictions = forest_reg.predict(housing_prepared)
    #     # forest_mse = mean_squared_error(housing_labels, housing_predictions)
    #     # forest_rmse = np.sqrt(forest_mse)
    # """SVector"""
    #     # svc.fit(housing_prepared, housing_labels)
    #     # housing_predictions = svc.predict(housing_prepared)
    #     # svc_mse = mean_squared_error(housing_labels, housing_predictions)
    #     # svc_rmse = np.sqrt(svm_mse)

## (alternative) GSCV approach

In [ ]:
# Individual example GridSearchCV/RandomizedSearchCV
# >>> clf = GridSearchCV(algo_pipe, param_grid, cv=5, verbose=0, 
# >>>                    return_train_score=False, n_jobs=-1)

# >>> rscv = RandomizedSearchCV(algo_pipe, param_grid, cv=5, verbose=0, 
# >>>                           return_train_score=False, n_jobs=-1)

"""# add into grid search to expand"""
# RidgeClassifier(tol=1e-2, solver="sag")
# Perceptron(max_iter=50)
# # PassiveAggressiveClassifier(max_iter=50) ??
# KNeighborsClassifier(n_neighbors=10)

# # LinearSVC with L1-based feature selection
# # -- The smaller C, the stronger the regularization.
# # -- The more regularization, the more sparsity.
# LinearSVC(penalty='l1' dual=False, tol=1e-3)

# SGDClassifier(alpha=.0001, max_iter=50, penalty="elasticnet")

# #"NearestCentroid (aka Rocchio classifier)
# NearestCentroid()

# # Naive Bayes models
# MultinomialNB(alpha=.01)
# BernoulliNB(alpha=.01)
# ComplementNB(alpha=.1)

In [ ]:
# """EXAMPLE"""
# params = {'n_estimators':[10,50,100,250],
#           'max_depth':[5,10,20],
#           'class_weight':[None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]}

# forest_example = RandomForestClassifier(random_state=42)
# gs = GridSearchCV(forest_example, params, scoring='roc_auc', n_jobs=-1)
# gs.fit(X_train, y_train)

# print("Best set of Parameters",gs.best_params_)
# print("Best Score",gs.best_score_)

In [ ]:
# #numeric pipe
# nprep_pipe = Pipeline(
#     [("n_imp", SimpleImputer(strategy='median')),
#     ('n_ssscale', MinMaxScaler()),
#     ('n_collin_remove_90', CollinearRemover(collinear_cutoff=.90))])

# #categoric pipe
# cprep_pipe = Pipeline(
#     [('c_imp', SimpleImputer(strategy='constant',fill_value='missing')),
#     ('c_hash_enc', ce.HashingEncoder(drop_invariant=True)),
#     ('c_collin_remove_95', CollinearRemover(collinear_cutoff=.95))])


# ncols, ccols, tcols = \
# list(ncols.cols), list(ccols.cols), list(tcols.cols)

# #build heterogeneous prep pipeline 
# ncprep_pipe = \
# ColumnTransformer([('nprep', nprep_pipe, ncols),
#                    ('cprep', cprep_pipe, ccols)])

# # train pipeline
# # >>> ncprep_pipe.fit(trx_opt)

# # and prepare numeric/categoric data for ml 
# # >>>trx_nc_prepped = ncprep_pipe.transform(trx_opt)

In [ ]:
# # Create estimator pipeline
# pipe = Pipeline([('prep', ncprep_pipe),('clf', lr)])

# # Create space of algorithms and their hyperparameters
# param_grid = [{'clf': [lr],
#                'clf__penalty': ['l1', 'l2'],
#                'clf__C': np.logspace(0, 4, 10),
#                'clf__solver':['lbfgs', 'liblinear', 'sag', 'saga'],
#                'clf__class_weight':[None, 'balanced']},
#               {'clf':[tree],
#                'clf__criterion': ['gini', 'entropy'],
#                'clf__max_depth': [5,10,20,None],
#                'clf__class_weight': [None, 'balanced'],
#                'clf__min_samples_split': [2,5,10],
#                'clf__max_features': ['auto', 'sqrt', 'log2'],
#                'clf__class_weight': [None, 'balanced']},
#               {'clf': [forest],
#                'clf__n_estimators': [10, 50, 100, 200],
#                'clf__criterion': ['gini', 'entropy'],
#                'clf__max_depth': [5, 10, 20, None],
#                'clf__class_weight': [None, 'balanced'],
#                'clf__max_features': ['auto', 'sqrt', 'log2'],
#                'clf__bootstrap':[True,False]},
#               {'clf': [sgd],
#                'clf__class_weight':['balanced', None],
#                'clf__penalty': ['l1','l2'],
#                'clf__early_stopping': [True, False]}]

# # eval individual estimators:

# for metric, model in zip(eval_method, clf_models):
#     GridSearchCV(model, param_grid, cv=5, scoring=metric)\
#                         .fit(trx_nc_prepped, trainy)

#     print(f"Best parameters:\n{clf.best_params_}")
#     print(f"\nGridSearch Mean test scores:\n\
#     {clf.cv_results_['mean_test_score']}")
#     print(f"\nGridSearch std test scores:\n\
#     {clf.cv_results_['std_test_score']}")
#     # for mean, std, params in zip(means, stds, clf.cv_results_['params']):
#     #     print("%s (+/-%s) for %s" % (mean, std * 2, params))

In [ ]:
# rscv = RandomizedSearchCV(pipe, param_grid,
#                           cv=3, verbose=0,
#                           return_train_score=False, n_jobs=-1)
# # fit/train
# rscv.fit(trx_nc_prepped,trainy.values)
# #>>> print(f"\nBest model parameters:\n{rscv.best_estimator_.get_params()['clf']}")
# print(f'Best model-parameters:\n{rscv.best_params_}') # Alternative
# print(f'\nBest model score:\n{rscv.best_score_}')
# print(f'\nBest model feature Importances:\n\
# {rscv.best_estimator_.feature_importances_}')

# best_model = rscv.best_estimator_

In [ ]:
# rscvs=[]
# for metric in metric_str_names:
#     rscvs.append(
#             RandomizedSearchCV(
#                 algo_pipe, param_grid, 
#                 cv=3, verbose=0, scoring=metric,
#                 return_train_score=False, n_jobs=-1).fit(trx_nc_prepped,
#                                                          trainy.values))
# best_models = []
# for each in rscvs:
#     best_models.append(each.best_estimator_)

# # Let's look at the score of each hp-combo tested:
# >>> gscv_res = gscv.cv_results_
# >>> for mean_score, params in zip(gscv_res["mean_test_score"],
# >>>                               gscv_res["params"]):
    
# >>>     print(np.sqrt(-mean_score), params)

# >>> rscv_feature_importances = rnd_search.best_estimator_.feature_importances_
# >>> rscv_feature_importances